<a href="https://colab.research.google.com/github/fromakim/2021Election_Analysis/blob/main/data_prepreocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import itertools
import json

In [18]:
import pandas as pd

In [19]:
data = []

In [20]:
for i in range(5):
    f = open(f'./data{i:02}.json', encoding = 'utf-8')
    data.append(json.load(f))

In [26]:
data[0]['includes']['users']

[{'created_at': '2020-03-09T05:58:08.000Z',
  'description': '대충 살아볼게',
  'id': '1236893975488778241',
  'name': '크즈',
  'profile_image_url': 'https://pbs.twimg.com/profile_images/1263765872897699840/kfcba_dl_normal.jpg',
  'protected': False,
  'public_metrics': {'followers_count': 8,
   'following_count': 34,
   'listed_count': 0,
   'tweet_count': 1869},
  'url': '',
  'username': 'unkogej',
  'verified': False},
 {'created_at': '2020-12-29T17:35:56.000Z',
  'description': '그래도 해야지',
  'id': '1343974013232562176',
  'location': 'Republic of Korea',
  'name': '눈을불알임',
  'profile_image_url': 'https://pbs.twimg.com/profile_images/1455197950918139904/QLwQyGxd_normal.png',
  'protected': False,
  'public_metrics': {'followers_count': 2991,
   'following_count': 185,
   'listed_count': 2,
   'tweet_count': 2523},
  'url': '',
  'username': 'himzo_',
  'verified': False},
 {'created_at': '2018-08-18T12:52:02.000Z',
  'description': '취향계',
  'id': '1030799481199984640',
  'name': '춰츄',
  'p

In [27]:
users = data[0]['includes']['users']

In [37]:
user_uniquekeys = list(set(itertools.chain(*list(map(lambda x : list(x.keys()), users)))))

In [39]:
data[0]['includes'].keys()

dict_keys(['users', 'tweets', 'media'])